# XML Parser 
Following is the code for parsing the xml files containing metadata for the volumes identified through the Internet Archive.
Using the __search.csv__ file containing identifiers of the volumes we generated from the Internet Archive, we create a list of the volumes whose xml files we want to parse.The column _'xmlfiles'_ has the list of those volumes. Through the xml files we extract the logical page numbers(pageNumber), physical page numbers(leafNum) and leaf hand side(handSide).


In [1]:
import xml.etree.ElementTree as ET
import csv
import pandas as pd
import os
import urllib

with open("search.csv","r") as xmlfiles:
    reader=csv.DictReader(xmlfiles)
    l=[d['xmlfiles'] for d in reader]
    
handSide = []    
pageNum = []
filename = []
leafNum = []
master = []


KeyboardInterrupt: 

In [ ]:
for i in l:
    try:
        xml = urllib.request.urlopen('https://archive.org/download/' + i + '/' + i + '_' + 'scandata.xml')
        tree = ET.parse(xml)
        root = tree.getroot()

        for page in root[2].findall('page'):
            leafNum.append(int(page.attrib['leafNum']))
            handSide.append(page.find('handSide').text)

            page_dict = {}    
            page_dict['leafNum'] = int(page.attrib['leafNum'])
            if page.find('pageNumber') != None:
                page_dict['pageNum'] = page.find('pageNumber').text
            else:
                page_dict['pageNum'] = ''

            page_dict['handSide'] = page.find('handSide').text
            page_dict['filename'] = i + '_' + '%04d' %  page_dict['leafNum']  
            master.append(page_dict)

        with open('xml_metadata.csv', 'w') as csvfile:
            writer1 = csv.DictWriter(csvfile, fieldnames=['filename','leafNum','handSide','pageNum'],lineterminator='\n')
            writer1.writeheader()
            for row in master:
                writer1.writerow(row)
    except:
        print(i)